In [ ]:
import pandas as pd
import numpy as np

### Lecture des fichiers

In [ ]:
data_urls = {
    "Élus arrondissements" : "https://www.data.gouv.fr/fr/datasets/r/3b6b2281-b9d9-4959-ae9d-c2c166dff118",
    "Élus municipaux" : "https://www.data.gouv.fr/fr/datasets/r/d5f400de-ae3f-4966-8cb6-a85c70c6c24a",
    "Élus communautaires" : "https://www.data.gouv.fr/fr/datasets/r/41d95d7d-b172-4636-ac44-32656367cdc7",
    "Élus départementaux" : "https://www.data.gouv.fr/fr/datasets/r/601ef073-d986-4582-8e1a-ed14dc857fba",
    "Élus régionaux" : "https://www.data.gouv.fr/fr/datasets/r/430e13f9-834b-4411-a1a8-da0b4b6e715c"
}

dfs = {}

for data_name,url in data_urls.items():
    dfs[data_name] = pd.read_csv(url, sep=";", dtype="str")

> Commentaires : Pour échapper cette erreur :

> C:\Users\lawan\AppData\Local\Temp\ipykernel_6288\1067538451.py:12: DtypeWarning: Columns (0,3,4) have mixed types. Specify dtype option on import or set low_memory=False. dfs[data_name] = pd.read_csv(url, sep=";")
> C:\Users\lawan\AppData\Local\Temp\ipykernel_6288\1067538451.py:12: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False. dfs[data_name] = pd.read_csv(url, sep=";")

> J'ai rajouté dtype = "str". Ainsi, au chargement, tous les colonnes sera considéré comme des strings. Pendant le netttoyage des données, je mettrai les types adéquats.



In [ ]:
for name, data in dfs.items():
    print(name,"\n\n",data.head(5))

### Identification des colonnes en commun

In [ ]:
colonnes_liste = {}
for name, df in dfs.items():
    for column in df.columns:
        # Récupérer la valeur de column si elle est déjà dans colonnes_liste, ou lui assigner la valeur {"nom_colonne": column, "occurence_colonne": 1, "fichiers": [name]} par défaut
        colonne_info = colonnes_liste.setdefault(column, {"nom_colonne": column, "occurence_colonne": 1, "fichiers": [name]})
        
        # Rajouter le fichier si il n'est pas encore compté, si non, rien
        if name not in colonne_info["fichiers"]:
            colonne_info["occurence_colonne"] += 1
            colonne_info["fichiers"].append(name)

In [ ]:
for keys,value in colonnes_liste.items():
    print(keys, " : ", value)

In [ ]:
# Transformation en dataFrame pandas
colonnes_final = []

for value in colonnes_liste.values():
    new_value = value.copy() # Afin de modifier sans affecter l'original
    new_value["fichiers"] = ",".join(new_value["fichiers"])
    colonnes_final.append(new_value)

colonnes_final = pd.DataFrame(colonnes_final)

print(colonnes_final.head(5))

In [ ]:
# Colonnes communes
colonnes_final[colonnes_final["occurence_colonne"]==5]["nom_colonne"]

> Commentaires : Il semble que les noms des colonnes soit propres. D'une fichier à une autre, il semble ne pas avoir des colonnes, a-priori identiques qui porterait des noms différents.

> **Les colonnes communes** au 5 jeu de données sont :  Nom de l'élu, Prénom de l'élu, Code sexe, Date de naissance, Code de la catégorie socio-professionnelle, Libellé de la catégorie socio-professionnelle, Date de début du mandat, Libellé de la fonction, Date de début de la fonction.

### Fusion des fichiers

In [ ]:
dfs_concat = pd.concat(dfs,axis=0)
dfs_concat = dfs_concat.reset_index().rename(columns={"level_0":"Type d'élu"})
dfs_concat = dfs_concat.drop(columns=["level_1"])
print(dfs_concat.head())

### Evaluation du jeu de données

In [ ]:
dfs_concat.dtypes

In [ ]:
lignes = 0
for name,df in dfs.items():
    lignes += df.shape[0]
    print(name," : ",df.shape)
print("Nombre de lignes des jeux de données : ", lignes)

In [ ]:
print("Nombre de lignes des jeux de données après fusion : ",dfs_concat.shape)

In [137]:
# Nombre de NA par colonne pour chaque dataset
lignes = 0
for name,df in dfs.items():
    print("\n=======",name,"==================")
    print(df.isna().sum(axis=0))


======= Élus arrondissements ==================
Code du département                                0
Libellé du département                             0
Code de la commune                                 0
Libellé de la commune                              0
Libellé du secteur                                 0
Nom de l'élu                                       0
Prénom de l'élu                                    0
Code sexe                                          0
Date de naissance                                  0
Lieu de naissance                                 11
Code de la catégorie socio-professionnelle         0
Libellé de la catégorie socio-professionnelle      0
Date de début du mandat                            0
Libellé de la fonction                           626
Date de début de la fonction                     626
Code de la nuance politique                        0
Libellé de la nuance politique                     0
dtype: int64

======= Élus municipaux ============

In [ ]:
## Nombre de NA par colonnes après fusion

dfs_concat.isna().sum(axis=0)

In [ ]:
## Nombre de lignes contenant des NA
nb_lignes_NA = dfs_concat.isna().any(axis=1).sum()

print("Nombre de lignes contenant des NA : ", nb_lignes_NA)

> **Commentaires** : Toutes les lignes contient des NA

###  Nettoyage des données

- Les données manquantes qui sont apparus après la fusion des jeux de données seront conservés telles quelles car leur apparition est logique. Les unités d'observation n'étant pas spécifiquement les mêmes Ex. un élu municipal dans un cas et un élu de l'arrondissement dans l'autre.
- En ce qui concerne les données manquantes qui étaient déjà présentes, distinctement dans chaque jeu de données :
    + Je vais supprimer les colonnes *Code de la collectivité à statut particulier* et *Libellé de la collectivité à statut particulier* car presque la totalité des lignes (86%) est manquante.  (Elus municipaux)
    + En me basant sur la colonne *Code de la Commune* et les valeurs présentes pour  *Code du département* et *Libellé du département*, je vais identifier les valeurs de *Code du département et Libellé du département* qui sont manquantes. Si cela ne suffit pas à les identifier, j'utiliserai le fichier de l'INSEE qui recense les départements. (Elus municipaux)
    + En ce qui concerne, les valeurs de *libellé de la fonction* qui sont manquantes pour les élus municipaux, je vais analyser les fonctions qu'occupent les valeurs qu'il prend pour les personnes dont la catégorie professionnelle est renseignée. En fonction des résultats, je ferai ou non une imputation. Je ferai de même pour *libellé de la fonction dans élus régionaux.
    